In [1]:
from mctnet.actions import locate_peaks
from mctnet.data_loading import _load_point_data
import torchio as tio
import numpy as np
from scipy import spatial
import napari

In [2]:
mct = tio.ScalarImage('./dataset/fiddler/cropped_with_csv_labs/test_images_whole_10/dampieri_male_16-image.nii.gz')
prediction = tio.ScalarImage('./output/dampieri_male_16-image.prediction_sigma_3.nii.gz')

In [3]:
prediction_locations = locate_peaks(
    './output/dampieri_male_16-image.prediction_sigma_3.nii.gz',
    save=True,
    plot=False,
    peak_min_dist=4,
    peak_min_val=0.5,
)

Locating peaks...
Saving peaks...


In [4]:
ground_truth_locations=np.loadtxt(
    './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
    delimiter=',',
    dtype=np.float
).astype(int).T

In [5]:
viewr = napari.view_points(ground_truth_locations, name='ground truths')
viewr.add_points(prediction_locations, name='prediction locations')
viewr.add_image(mct.numpy(), name='mct')
viewr.add_image(prediction.numpy(), name='prediction heatmap')

/home/jake/projects/mctnet/venv/lib/python3.8/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'prediction heatmap' at 0x7f7976cceeb0>

In [16]:
tree = spatial.KDTree(prediction_locations)
closest_dists_1_and_2, closest_neighbours_1_and_2 = tree.query(ground_truth_locations, k=2)

closest_dists = np.array([x[0] for x in closest_dists_1_and_2])
second_closest_dists = np.array([x[1] for x in closest_dists_1_and_2])
closest_neighbours = np.array([x[0] for x in closest_neighbours_1_and_2])
second_closest_neighbours = np.array([x[1] for x in closest_neighbours_1_and_2])

is_within_distance = closest_dists < 6
correct_prediction_distances = closest_dists[is_within_distance]

correct_prediction_coord_indx = closest_neighbours[is_within_distance]
all_indxs = np.arange(0, len(prediction_locations))
incorrect_prediction_coord_indx = [x for x in all_indxs if x not in correct_prediction_coord_indx]

correct = prediction_locations[correct_prediction_coord_indx]
false_positives = prediction_locations[incorrect_prediction_coord_indx]

In [17]:
viewr.add_points(correct, name='correctly predicted corneas based on within distance')
viewr.add_points(false_positives, name='false positive predicted corneas based on within distance')

<Points layer 'false positive predicted corneas based on within distance [1]' at 0x7f7989c1a880>

In [18]:
# print summary of evaluation
print('Evaluation summary:')
print('\tPercentage correctly predicted:', len(correct) / len(ground_truth_locations) * 100)
print('\tNumber correctly predicted:', len(correct))
print('\tAverage distance (of correct predictions):', np.mean(correct_prediction_distances))
print('\tMedian distance (of correct predictions):', np.median(correct_prediction_distances))
print('\tStandard deviation (of correct predictions):', np.std(correct_prediction_distances))
print('\tMinimum distance (of correct predictions):', np.min(correct_prediction_distances))
print('\tMaximum distance (of correct predictions):', np.max(correct_prediction_distances))
print('\tNumber missing:', len(ground_truth_locations) - len(correct))
print('\tNumber of false positives:', len(false_positives))
print('\tNumber in ground truth:', len(ground_truth_locations))
print('\tNumber in prediction:', len(prediction_locations))

Evaluation summary:
	Percentage correctly predicted: 88.39642381586457
	Number correctly predicted: 4647
	Average distance (of correct predictions): 2.4498265422540166
	Median distance (of correct predictions): 2.449489742783178
	Standard deviation (of correct predictions): 0.7374707771922006
	Minimum distance (of correct predictions): 0.0
	Maximum distance (of correct predictions): 5.916079783099616
	Number missing: 610
	Number of false positives: 1200
	Number in ground truth: 5257
	Number in prediction: 5843


In [19]:
correct

array([[483, 589, 179],
       [498, 590, 195],
       [498, 594, 203],
       ...,
       [173, 412, 502],
       [176, 405, 510],
       [178, 399, 519]])

In [21]:
# def estimate_error(peaks, ground_truth_peaks):
#     breakpoint()
#     return None


# def test_different_peak_params(prediction_path, ground_truth_coords, min_distances, min_vals, plot_and_pause=False):
#     """Estimate the error of the peaks with different peak-finding parameters.
    
#     Args: 
#         prediction_path (str): Path to the prediction volume.
#         ground_truth_coords (list): List of ground truth coordinates (x, y, z).
#         min_distances (list): The list of minimum distances you want to try.
#         min_vals (list): The list of minimum values (float) you want to try.
#         plot_and_pause (bool): If True, plot the results and pause after each parameter error measurement.
#         If False, don't plot and don't pause.

#     Returns:
#         list: The list of errors.
#     """
#     if type(min_distances) == int:
#         min_distances = [min_distances]
#     if type(min_vals) == float:
#         min_vals = [min_vals]

#     errors = []
#     for min_distance in min_distances:
#         for min_val in min_vals:
#             peaks = locate_peaks(
#                 prediction_path,
#                 save=True,
#                 plot=plot_and_pause,
#                 peak_min_dist=min_distance,
#                 peak_min_val=min_val
#             )

#             errors.append(estimate_error(peaks, ground_truth_coords))

#     return errors


In [22]:
# errors_dampieri = test_different_peak_params(
#     prediction_path='/home/jake/projects/mctnet/dataset/fiddler/cropped/test_images_whole/dampieri_male_16-image.nii.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=8,
#     min_vals=0.3,
#     # min_distances=[4, 8, 12],
#     # min_vals=[0.2, 0.3, 0.4, 0.5]
# )
# errors_flammula = test_different_peak_params(
#     prediction_path='/home/jake/projects/mctnet/dataset/fiddler/cropped/test_images_whole/flammula_20190925_male_left-image.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         '',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=[4, 8, 12],
#     min_vals=[0.2, 0.3, 0.4, 0.5]
# )

In [37]:
# # testing only
# peaks, ground_truth_coords = test_different_peak_params(
#     prediction_path='./dataset/fiddler/cropped/test_images_whole/dampieri_male_16-image.nii.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=8,
#     min_vals=0.3,
# )


Locating peaks...
Saving peaks...
